In [1]:
import pandas as pd
import torch
import lightning as L

from model.modeling_demolta import DeMOLTaConfig
from trainer import LitMOLAForRegression, SaveTrainableParamsCheckpoint
from datautils import LitMOLAFintTuneDataModule

In [2]:
BATCH_SIZE = 4
SEED = 42
TEXT_MODEL_NAME = 'facebook/galactica-125m'

In [3]:
L.seed_everything(SEED)

Global seed set to 42


42

In [4]:
lit_finetune_data_module = LitMOLAFintTuneDataModule(
    df_path='./data/train.csv',
    batch_size=BATCH_SIZE,
    seed=SEED,
    k_fold=5,
    train_fold=0,

)

In [5]:
demolta_config = DeMOLTaConfig(
    num_layers=12,
    hidden_dim=384,
    ff_dim=1536,
    num_heads=6,
)

In [6]:
lit_model = LitMOLAForRegression(
    demolta_config=demolta_config,
    text_model_name=TEXT_MODEL_NAME,
    n_class=2,
)

In [7]:
checkpoint = torch.load('./checkpoint/mola-pretrain-step=890000-val_loss=1.38.ckpt')
lit_model.load_state_dict(checkpoint, strict=False)

_IncompatibleKeys(missing_keys=['model.regressor.0.weight', 'model.regressor.0.bias', 'model.regressor.1.weight', 'model.regressor.1.bias', 'model.regressor.3.weight', 'model.regressor.3.bias'], unexpected_keys=[])

In [8]:
checkpoint_callback = SaveTrainableParamsCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoint/',
    filename='mola-pretrain=890000-finetune+{epoch}-{val_loss:.4f}',
    save_top_k=3,
)

In [9]:
trainer = L.Trainer(
    accelerator='gpu',
    max_epochs=10,
    callbacks=[checkpoint_callback],
    val_check_interval=0.5,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(lit_model, lit_finetune_data_module)

You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading some Jax models, missing a dependency. No module named 'jax'
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\dust\Documents\Dacon\med\demolta\checkpoint exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type 

Sanity Checking: 0it [00:00, ?it/s]

c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]